In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\amil\\OneDrive\\Documents\\Data_Science_end_to_end_project'

In [2]:
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

from src.constants import *
from src.utils.common import read_yaml, create_directories
from src import logger


In [3]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [5]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [6]:
data.shape

(1599, 12)

In [7]:
@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    

In [10]:
class ConfigurationManager:
    def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH,
                schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        
    def get_data_validation_config(self)-> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )
        return data_validation_config


In [11]:
import pandas as pd
import logging
from pathlib import Path

logger = logging.getLogger(__name__)

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self):
        """
        Validate:
        1. If all required columns from schema are present.
        2. If each column has the correct data type.

        Returns:
            bool: True if validation passed, False otherwise.
        """
        try:
            validation_status = True  # Assume valid until proven otherwise

            # Load CSV data
            data = pd.read_csv(self.config.unzip_data_dir)
            data_columns = list(data.columns)
            schema_columns = self.config.all_schema  # Expected schema

            logger.info("Starting column and dtype validation.")

            # Iterate through schema to validate columns
            for col, expected_dtype in schema_columns.items():

                # 1. Check if column exists
                if col not in data_columns:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Failed: Missing column -> {col}\n")
                    logger.error(f"Missing expected column: {col}")
                    return validation_status  # Stop further checks immediately

                # 2. Check if column's dtype matches
                actual_dtype = str(data[col].dtype)
                if actual_dtype != expected_dtype:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(
                            f"Validation Failed: Column '{col}' has dtype '{actual_dtype}', "
                            f"expected '{expected_dtype}'\n"
                        )
                    logger.error(
                        f"Column '{col}' dtype mismatch: expected {expected_dtype}, got {actual_dtype}"
                    )
                    return validation_status  # Stop further checks immediately

            # If all validations pass
            with open(self.config.STATUS_FILE, "w") as f:
                f.write("Validation Passed: All columns and data types are correct.\n")

            logger.info("Data validation passed successfully.")
            return validation_status

        except Exception as e:
            logger.exception("An error occurred during data validation.")
            raise e


In [12]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config= data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e
    

[2025-09-25 13:57:00,764:, INFO:, common:, yaml file : config\config.yaml loaded successfully]
[2025-09-25 13:57:00,776:, INFO:, common:, yaml file : schema.yaml loaded successfully]
[2025-09-25 13:57:00,781:, INFO:, common:, created directory for artifacts/data_validation]
[2025-09-25 13:57:00,898:, INFO:, 3479964307:, Starting column and dtype validation.]
[2025-09-25 13:57:00,915:, INFO:, 3479964307:, Data validation passed successfully.]
